<a href="https://colab.research.google.com/github/dizzySummer/NLP-TensorFlow2.0/blob/master/amazonwbpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/My Drive/amazon-fine-food-reviews/')

import attention


In [3]:
import io
import os
import pandas as pd
import numpy as np
#import data that stored on Google drive
import io
df = pd.read_csv('/content/drive/My Drive/amazon-fine-food-reviews/Reviews.csv')
# Dataset is now stored in a Pandas Dataframe
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
# this block will be needed in the real dataset
df=df.dropna()
df=df[['Summary','Text']].reset_index(drop=True)
df.head()

len(df) #568411

568411

In [5]:
print("preprocessing")
df["Text"]=df["Text"].str.lower()
df["Summary"]=df["Summary"].str.lower()

df.head()

preprocessing


,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,"""delight"" says it all",this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price. there was a wid...


In [6]:
!pip install sentencepiece

In [7]:
import logging
import sentencepiece as spm
import os

logging.basicConfig(level=logging.INFO)
input_file="to_mytxt2.txt"
vocab_size = 15000
prefix = 'm_bpe'


templates = '--input={} --model_prefix={} --vocab_size={} --pad_id=0 --unk_id=1 --bos_id=2  --eos_id=3 --model_type=bpe'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.train(cmd)
print("BPE model trained!")


BPE model trained!


In [8]:
# makes segmenter instance and loads the model file (m.model)
sp_bpe = spm.SentencePieceProcessor()
sp_bpe2 = spm.SentencePieceProcessor()
sp_bpe.load('{}.model'.format(prefix))
sp_bpe2.load('{}.model'.format(prefix))
print("Trained BPE model loaded")


#option={"unk": "UNK","eos": "</s>"}
extra_options = 'bos:eos' #'reverse:bos:eos'
sp_bpe.SetEncodeExtraOptions(extra_options)

print("sp_bpe with extra options ")  
print(sp_bpe.EncodeAsIds('so good'))
print(sp_bpe.EncodeAsPieces('so good'))

#print("sp_bpe2 without extra options ")
#print(sp_bpe2.EncodeAsIds('so good'))
#print(sp_bpe2.EncodeAsPieces('so good'))

print(sp_bpe.DecodeIds([5]))
# vocabulary size
print(len(sp_bpe))

Trained BPE model loaded
sp_bpe with extra options 
[2, 115, 139, 3]
['<s>', '▁so', '▁good', '</s>']
a
15000


In [0]:
df.to_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/to_mytxt2.txt', header=None, index=None, sep=' ', mode='a')


In [10]:
import warnings
warnings.filterwarnings("ignore")

df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_pieces(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe2.encode_as_pieces(x))
#val.head(10)


df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[<s>, ▁good, ▁quality, ▁dog, ▁food, </s>]","[▁i, ▁have, ▁bought, ▁several, ▁of, ▁the, ▁vit..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[<s>, ▁not, ▁as, ▁advertised, </s>]","[▁product, ▁arrived, ▁labeled, ▁as, ▁jumbo, ▁s..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[<s>, ▁"", delight, "", ▁says, ▁it, ▁all, </s>]","[▁this, ▁is, ▁a, ▁confection, ▁that, ▁has, ▁be..."
3,cough medicine,if you are looking for the secret ingredient i...,"[<s>, ▁cough, ▁medicine, </s>]","[▁if, ▁you, ▁are, ▁looking, ▁for, ▁the, ▁secre..."
4,great taffy,great taffy at a great price. there was a wid...,"[<s>, ▁great, ▁taffy, </s>]","[▁great, ▁taffy, ▁at, ▁a, ▁great, ▁price, ., ▁..."


In [11]:
df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_ids(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe2.encode_as_ids(x))
#val.head(10)
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 139, 582, 335, 213, 3]","[6, 103, 493, 972, 46, 10, 1258, 542, 1486, 33..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 94, 119, 2638, 3]","[167, 1107, 5115, 119, 6697, 4041, 2848, 293, ..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 32, 8841, 14969, 1303, 41, 173, 3]","[62, 48, 5, 8501, 86, 215, 373, 975, 5, 635, 4..."
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 6684, 9004, 3]","[203, 79, 104, 747, 65, 10, 5407, 532, 52, 305..."
4,great taffy,great taffy at a great price. there was a wid...,"[2, 172, 4967, 3]","[172, 4967, 174, 5, 172, 300, 14968, 308, 108,..."


In [12]:
type(df['Tokened_Summary'][0])

list

In [13]:
df['summSum']=df['Tokened_Summary'].str.len()
df['summSum'].describe()

count    568411.000000
mean          7.150532
std           3.349560
min           3.000000
25%           5.000000
50%           6.000000
75%           9.000000
max          90.000000
Name: summSum, dtype: float64

In [14]:
df['textSum']=df['Tokened_Text'].str.len()
df['textSum'].describe()

count    568411.000000
mean        100.951268
std         105.346128
min           3.000000
25%          41.000000
50%          69.000000
75%         122.000000
max        4766.000000
Name: textSum, dtype: float64

In [15]:
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text,summSum,textSum
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 139, 582, 335, 213, 3]","[6, 103, 493, 972, 46, 10, 1258, 542, 1486, 33...",6,54
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 94, 119, 2638, 3]","[167, 1107, 5115, 119, 6697, 4041, 2848, 293, ...",5,38
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 32, 8841, 14969, 1303, 41, 173, 3]","[62, 48, 5, 8501, 86, 215, 373, 975, 5, 635, 4...",8,122
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 6684, 9004, 3]","[203, 79, 104, 747, 65, 10, 5407, 532, 52, 305...",4,49
4,great taffy,great taffy at a great price. there was a wid...,"[2, 172, 4967, 3]","[172, 4967, 174, 5, 172, 300, 14968, 308, 108,...",4,32


In [16]:
len(df)

568411

In [0]:
#https://stackoverflow.com/questions/18936957/count-distinct-words-from-a-pandas-data-frame
#df['Tokened_Text'].str.lower().str.split()

In [18]:
#remove columns which length of tokened_summary > 15 and length of tokened_text >60
df = df[(df['textSum'] <100)&(df['summSum']<9)].reset_index(drop=True)
len(df)

305941

In [19]:
df=df[['Tokened_Text','Tokened_Summary']]
df.head()

,Tokened_Text,Tokened_Summary
0,"[6, 103, 493, 972, 46, 10, 1258, 542, 1486, 33...","[2, 139, 582, 335, 213, 3]"
1,"[167, 1107, 5115, 119, 6697, 4041, 2848, 293, ...","[2, 94, 119, 2638, 3]"
2,"[203, 79, 104, 747, 65, 10, 5407, 532, 52, 305...","[2, 6684, 9004, 3]"
3,"[172, 4967, 174, 5, 172, 300, 14968, 308, 108,...","[2, 172, 4967, 3]"
4,"[6, 543, 5, 2991, 2871, 65, 4967, 29, 648, 62,...","[2, 489, 4967, 3]"


In [20]:
len(df)

305941

In [21]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.3)
train.shape
#val.shape
train.head()

,Tokened_Text,Tokened_Summary
48538,"[337, 1202, 143, 79, 548, 293, 222, 5980, 7695...","[2, 88, 442, 1981, 3]"
19671,"[10, 1988, 847, 103, 373, 329, 3342, 1002, 29,...","[2, 1557, 8089, 3]"
151195,"[768, 1387, 29, 300, 14968, 4712, 12506, 6708,...","[2, 491, 3594, 104, 172, 3]"
235054,"[62, 48, 1626, 337, 6, 1180, 115, 86, 6, 436, ...","[2, 489, 14977, 3]"
265458,"[62, 108, 1626, 337, 6, 1598, 65, 88, 6282, 13...","[2, 537, 65, 13831, 1641, 638, 14977, 3]"


In [22]:
len(train)

214158

In [23]:
val.head()

,Tokened_Text,Tokened_Summary
235450,"[6, 222, 62, 501, 14968, 105, 522, 103, 5, 536...","[2, 172, 501, 3]"
179861,"[191, 115, 79, 514, 14972, 79, 143, 925, 238, ...","[2, 1161, 2105, 3]"
40631,"[10, 167, 48, 172, 14968, 6, 143, 14973, 14949...","[2, 2080, 2100, 3]"
279870,"[163, 104, 3451, 14977, 4105, 694, 472, 65, 22...","[2, 768, 14977, 3]"
133169,"[88, 491, 1071, 10, 2221, 29, 108, 1137, 33, 2...","[2, 491, 213, 3]"


In [24]:
len(val)

91783

In [25]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs))) # 2000
print(sp_bpe.unk_id())
for id in range (4):
  print(sp_bpe.id_to_piece(id),sp_bpe.is_control(id))

num of vocabs = 15000
1
<pad> True
<unk> False
<s> True
</s> True


In [26]:
x_train=train.iloc[:,0]
x_val=val.iloc[:,0] 
y_train=train.iloc[:,1]
y_val=val.iloc[:,1]
x_train.head()

48538     [337, 1202, 143, 79, 548, 293, 222, 5980, 7695...
19671     [10, 1988, 847, 103, 373, 329, 3342, 1002, 29,...
151195    [768, 1387, 29, 300, 14968, 4712, 12506, 6708,...
235054    [62, 48, 1626, 337, 6, 1180, 115, 86, 6, 436, ...
265458    [62, 108, 1626, 337, 6, 1598, 65, 88, 6282, 13...
Name: Tokened_Text, dtype: object

In [27]:
from keras.preprocessing.sequence import pad_sequences

#padding zero up to maximum length
x_train   =   pad_sequences(x_train,  maxlen=100, padding='post')
x_val   =   pad_sequences(x_val, maxlen=100, padding='post')
#padding zero up to maximum length
y_train    =   pad_sequences(y_train,  maxlen=9, padding='post')
y_val   =   pad_sequences(x_val, maxlen=9, padding='post')

print(x_train[14])

Using TensorFlow backend.


[   62   360  1541  4304    91   131 14968     6   103  1288   710    41
    52  1186  7838  1038    46  1112   360 14968    41   215     5   219
  1198  1990   209   592   360   387   853  1374  2148   478   341 14968
    41   340   215     5  1156  3267   168    33    41    86    48   193
  3083 14968   378   704    41    48     5   219  1198   168     6   103
   373   710   640    46    41   209    10   360     6   156  6836    41
    91 14968    10   965   340  6436   341    52   429  2764 14968     6
  1442 14973 14949  1190   344  1373    91    41 14972  1421     6  1608
    33  1717 14968     0]


In [28]:
#we get total unique text tokens x_train =14698
x_voc=0
x_tokens=[]
for row in x_train:
  for tokenx in row:
    if tokenx not in x_tokens:
        x_tokens.append(tokenx)
        #print(token, end=' ')
        x_voc+=1

x_voc=x_voc+1
print(x_voc)


14294


In [29]:
x_voc_size=len(sp_bpe)
print(x_voc_size)

15000


In [30]:
#make a comparision between this one and next cell. 
#we get total unique text tokens y_train =10866
y_voc=0
y_tokens=[]

for row in y_train:
  for tokeny in row:
    if tokeny not in y_tokens:
      y_tokens.append(tokeny)
     
      y_voc+=1

y_voc=y_voc+1 

print(y_voc)

9934


In [31]:
y_train.shape

(214158, 9)

In [32]:
x_train.shape

(214158, 100)

In [33]:

#we get total unique text tokens y_train =15000 /actually this is the whole voc list size
y_voc2=0
y_tokens2=[]
t = 0
for row in y_train:
  import numpy as np
  t = max(t,np.max(row))
  for token in row:
    if token not in y_tokens2:
      y_tokens2.append(token)
      #print(token,end='')
      y_voc2+=1

y_voc2=y_voc+1 
y_voc2 = t+1

y_voc2_size=len(sp_bpe)#corrected from sp_bpe2 to sp_bpe
print(y_voc2_size)


15000


In [0]:
#import csv

#train_csv = train.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv', index = None, header=True)
#val_csv = val.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv', index = None, header=True)



In [34]:

print(len(train))
print(len(val))


214158
91783


In [0]:
#df = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv')
##Dataset is now stored in a Pandas Dataframe

#df_v = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv')
#len(df_v)

In [36]:
import tensorflow as tf

#from keras.preprocessing.text import Tokenizer 
#from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


In [0]:
import time
from tensorflow import keras
import datetime, os

#import datetime, os
from tensorflow.python.keras.callbacks import TensorBoard


In [38]:
from keras import backend as K
from tensorflow.keras.layers import *
from attention import *
K.clear_session()

latent_dim = 300
embedding_dim=100  # output vector from embedding layer

#encoder

max_text_len=100

encoder_inputs = Input(shape=(max_text_len,)) 
encoder_embedding = Embedding(x_voc_size, embedding_dim, trainable=True)(encoder_inputs)


#encoder_lstm1
encoder_LSTM1=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output1,state_h1,state_c1=encoder_LSTM1(encoder_embedding)

#encoder_lstm2
encoder_LSTM2=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2,state_h2,state_c2=encoder_LSTM2(encoder_output1)

#encoder_lstm3
encoder_LSTM3=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs,state_h,state_c=encoder_LSTM3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc2_size, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.4)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc2_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1500000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [0]:
#Can I use TensorBoard with Google Colab?
#Tensorboard: ValueError: Duplicate plugins for name projector #22676
#https://github.com/pytorch/pytorch/issues/22676

logs_base_dir = "./logs"
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)



In [41]:
import os
if os.path.isfile("model_weights.h5"):
    print("Load the model")
    model.load_weights('/content/drive/My Drive/modelsone/model_weights.h5')
else:
  print("Start to train the model")
  model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=8)

  history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=5,callbacks=[es, tensorboard_callback],batch_size=384, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))
  model.save_weights('/content/drive/My Drive/modelbpe/model_weights.h5')

Start to train the model
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 214158 samples, validate on 91783 samples
Epoch 1/5
214158/214158 [==============================] - 1130s 5ms/sample - loss: 2.7564 - val_loss: 0.5111
Epoch 2/5
214158/214158 [==============================] - 1126s 5ms/sample - loss: 2.3533 - val_loss: 0.5143
Epoch 3/5
214158/214158 [==============================] - 1129s 5ms/sample - loss: 2.1998 - val_loss: 0.6664
Epoch 4/5
214158/214158 [==============================] - 1128s 5ms/sample - loss: 2.1090 - val_loss: 0.5360
Epoch 5/5
214158/214158 [==============================] - 1125s 5ms/sample - loss: 2.0433 - val_loss: 0.3001


In [0]:
#create normal h5 file

model.save('/content/drive/My Drive/modelbpe/model.h5') 

In [43]:
mytest = sp_bpe.encode_as_ids('love this bbq sauce... can\'t wait for this to be more available, i buy it up if i see a few at the local stores.')
newtest= sp_bpe.encode_as_ids('and I feel like a new man with this bag I looooove it !!')
print(mytest)
test = pad_sequences([newtest],  maxlen=100, padding='post')
print(test)
secNewTest=sp_bpe.DecodeIds(newtest)
#?? means UNK
print(secNewTest)

[2, 222, 62, 1961, 831, 293, 143, 14973, 14949, 1694, 65, 62, 33, 58, 219, 1065, 14972, 6, 257, 41, 265, 203, 6, 404, 5, 635, 174, 10, 780, 907, 14968, 3]
[[    2    29 14947     1   733   129     5   562  1244    91    62   317
  14947     1   110  5868    45    41  7213     3     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
and  ⁇  feel like a new man with this bag  ⁇  looooove it !!


In [0]:

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

encoder_model.save_weights('/content/drive/My Drive/modelbpe/encoder_model_weights.h5')

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])
decoder_model.save_weights('/content/drive/My Drive/modelbpe/decoder_model_weights.h5')

max_summary_len = 9
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = 2 #target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))
        sampled_token = sp_bpe.DecodeIds([sampled_token_index]) #reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!=3):#'eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token_index == 3 or sampled_token_index == 0 or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence
  
#decode_sequence(test)

In [0]:
#define the functions to convert an integer sequence to a word sequence for summary as well as the reviews:

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_seq[2]) and i!=target_seq[3]):
            newString=newString+sp_bpe.DecodeIds[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+sp_bpe2.DecodeIds[i]+' '
    return newString

In [46]:
!pip install rouge

In [47]:
#test some examples
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.5238095189484127, 'p': 0.6285714285714286, 'r': 0.4489795918367347}, 'rouge-2': {'f': 0.2727272680991736, 'p': 0.375, 'r': 0.21428571428571427}, 'rouge-l': {'f': 0.4517704517703128, 'p': 0.5714285714285714, 'r': 0.40816326530612246}}]


In [48]:
for i in range(200,300):
    print("Review:",sp_bpe2.DecodeIds(x_train[i].tolist()))
    print("Original summary:",sp_bpe.DecodeIds(y_train[i].tolist()))
    print("Predicted summary:", decode_sequence(x_train[i].reshape(1,max_text_len)))
 
    #ref. https://pypi.org/project/rouge/
    rouge=Rouge()
    scores = rouge.get_scores(decode_sequence(x_train[i].reshape(1,max_text_len)),sp_bpe.DecodeIds(y_train[i].tolist()),avg=True)
    print(scores)
  
    print("\n")

Review: i used a manual ceramic ,conical drip coffee maker with a  ⁇ 4 melitta filter, which i have found, makes the best coffee i have ever tasted. this coffee was average tasting, medium bodied and had a slight bitterness, but not offensive. i will not buy it again.
Original summary: average flavor
Predicted summary:  not for me 
{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}


Review: i found these pretzels originally when i started eating a primarily gluten-free diet. they taste great! i had been getting them from my grocery store, but they are rather expensive, so i was thrilled to find them on amazon for a less expensive price and i was able to get them for even less when i got them on auto-ship. these pretzels are my main snack food and even my family loves them. i would highly recommend them.
Original summary: great pretzels!
Predicted summary:  great snack 
{'rouge-1': {'f': 0.4999999950000001, 

In [50]:
%tensorboard --logdir logs
#click "Graphs" on the menu bar and take a look those generated grpahs

UsageError: Line magic function `%tensorboard` not found.


In [0]:

![This image is just show effect of tensorboard.](https://drive.google.com/file/d/1ak1cPo4oTgbYKMOkz0JxBw1QwQ9bJNIf/view?usp=sharing)